In [ ]:
%reload_ext autoreload
%autoreload 2

### Define the Stieltjes transform $t_r(z)$ as in Peixoto 2013, Spectra of random modular graphs and the symbols

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from numpy.random import random as rand

import networkqit as nq
import networkx as nx

from networkqit import graph_laplacian as GL
from networkqit import normalized_graph_laplacian as NGL

import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import sympy as sp
import sympyprinting as spp
from sympy import init_printing

import mpmath as mp
import networkx as nx
from etaprogress.progress import ProgressBar

## Define utility functions to generate networks

In [ ]:
def planted_partition_graph(n, b, pin, pout):
    nb = int(n / b)
    A = (np.random.random((n, n)) < pout).astype(float)
    for i in range(0, b):
        T = np.triu((np.random.random((nb, nb)) < pin).astype(float))
        T = T + T.T
        A[i * nb:(i + 1) * nb, i * nb:(i + 1) * nb] = T

    np.fill_diagonal(A, 0)
    A = np.triu(A)
    A = A + A.T
    return A

def hierarchical_random_graph2(ers, nr):
    N = np.sum(nr)
    b = len(ers)
    nrns = np.reshape(np.kron(nr, nr), [b, b])
    M = ers / nrns
    A = np.zeros([N, N])
    idx = np.cumsum([0] + nr)
    for i in range(0, b):
        ri = np.array(range(idx[i], idx[i + 1] + 1))
        for j in range(0, b):
            rj = np.array(range(idx[j], idx[j + 1] + 1))
            R = np.random.random([len(ri) - 1, len(rj) - 1])
            #R = np.triu(R)
            #R += R.T
            A[ri.min():ri.max(), rj.min():rj.max()] = (nrns[i, j] * R) < ers[i, j]
    A = np.triu(A, 1)
    A += A.T
    return A

def hierarchical_random_graph(sigma2rs, nr):
    N = np.sum(nr)
    A = np.zeros([N, N])
    b = len(sigma2rs)
    idx = np.cumsum([0] + nr)
    for i in range(0, b):
        ri = np.array(range(idx[i], idx[i + 1] + 1))
        for j in range(0, b):
            rj = np.array(range(idx[j], idx[j + 1] + 1))
            R = np.random.random([len(ri) - 1, len(rj) - 1])
            A[ri.min():ri.max(), rj.min():rj.max()] = R < sigma2rs[i, j]
    A = np.triu(A, 1)
    A += A.T
    return A


def gammadiff(a, z0, z1):  # this version of Gamma does not have infinite recursion problems
    return mp.gammainc(a, z0) - mp.gammainc(a, z1)

In [ ]:
def compute_tr(z, t0, ers, nr, matrix, tol=1E-9, maxsteps=50000):
    b = ers.shape[0]
    if len(nr) != b:
        raise Exception('Incompatible dimensions')
    # Their definition changes for every different submatrix
    sigma2rs, M = None, None
    nrns = np.reshape(np.kron(nr,nr),[b,b])
    er = ers.sum(axis=0)
    if matrix is 'laplacian':
        sigma2rs = ers/nrns
        M = -sigma2rs
    elif matrix is 'adjacency':
        sigma2rs = ers/nrns
        M = sigma2rs
    elif matrix is 'norm_laplacian':
        sigma2rs = ers/np.reshape(np.kron(er,er),[b,b])
        M = -ers/np.sqrt(np.multiply(nrns,ers))

    # Define the intrablock average degrees dr
    # since we work with undirected graphs
    dr = er/np.array(nr)

    Eq2 = [None] * b

    # This is the supercomplicated way to avoid shallow copy of lambda
    # functions around
    for r in range(0, b):
        if matrix is 'laplacian':
            Eq2[r] = lambda rr, T: (mp.power(dr[rr], (z - np.sum([sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b)]))) * gammadiff(np.sum([sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b)]) - z, 0, -dr[rr]) + T[
                                    rr] * mp.exp(dr[rr] + 1j * mp.pi * (np.sum([sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b)]) - z))) * mp.exp(-dr[rr] - 1j * mp.pi * (np.sum([sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b)]) - z))
        elif matrix is 'adjacency':
            Eq2[r] = lambda rr, T: T[rr] + 1 / (np.sum([ sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b) ]) - z)
        elif matrix is 'norm_laplacian':
            Eq2[r] = lambda rr, T: T[rr] + 1 / (np.sum([sigma2rs[rr, ss] * nr[ss] * T[ss] for ss in range(0, b)]) - z + 1)

    try:
        trsol = mp.findroot(lambda *t: [Eq2[rr](rr, [*t]) for rr in range(0, b)], x0=t0, solver='muller', maxsteps=maxsteps)
        return [trsol[r] for r in range(0, b)]
    except:
        raise Exception('Value of z was ' + str(z))
        return t0

############################################################
############################################################


def compute_detached_tr(ers, z0, t0, nr, matrix, eps=1E-7, maxsteps=50000):
    b = ers.shape[0]
    sigma2rs, M = None, None

    nrns = np.reshape(np.kron(nr,nr),[b,b])
    er = ers.sum(axis=1)
    if matrix is 'laplacian':
        sigma2rs = ers/nrns
        M = -sigma2rs
    elif matrix is 'adjacency':
        sigma2rs = ers/nrns
        M = sigma2rs
    elif matrix is 'norm_laplacian':
        sigma2rs = ers/np.reshape(np.kron(er,er),[b,b])
        M = -ers/np.sqrt(np.multiply(nrns,ers))

    IB = sp.eye(b)
    NB = sp.diag(*nr)

    def TzB(z):
        val = sp.diag(*compute_tr(z + eps * 1j, t0, ers, nr, matrix))
        return val

    def eq4(z):  # Equation 4 of Peixoto paper
        det = sp.simplify((IB - TzB(z) * M * NB).det())
        return mp.mpc(det)

    def trsum(z):
        tr = compute_tr(z + eps * 1j, t0, ers, nr, [sp.KroneckerDelta(sp.Symbol('c_', integer=True), 0)] * b)
        return np.sum([t.imag for t in tr])
    z_detached = mp.findroot(eq4, x0=mp.mpc(z0), tol=1E-9, solver='muller', maxsteps=maxsteps)
    return z_detached.real


def compute_rho(allz, ers, nr, matrix, eps=1E-7, maxsteps=150, include_isolated=False, t0=None):

    N = np.sum(nr)  # total number of nodes

    bar = ProgressBar(len(allz), max_width=40)

    b = ers.shape[0]
    if t0 is None:
        # use a default initial value for the stieltjes transform tr
        t0 = [mp.mpc('0')] * b

    print('Continuos band')
    io = open('data.dat','w')
    rho = np.zeros_like(allz)  # initialize rho as zeros
    for i, z in enumerate(allz):
        bar.numerator = i + 1
        t = compute_tr(z + eps * 1j, t0, ers, nr, matrix)
        for tt in t:
            io.write('%g %g ' % (float(tt.real),float(tt.imag)))
        io.write('\n')
        t0 = t # initialize from previous solution
        rho[i] = -(1.0 / (N * np.pi)) * np.sum([nr[r] * t[r].imag for r in range(0, b)])
        print('\r', bar, end='')
        #print('\r%s Percent done=%.1f %%\tz=%.2f\trho(z)=%g ' % (bar, float(i+1)/len(allz)*100, z, rho[i]), end='')
    io.close()
    # Then look for the detached eigenvalues by looking for the z such that it solves Eq.4=0
    # where rho is 0 look for separate eigenvalues
    if include_isolated:
        allzlist = allz.tolist()
        rholist = rho.tolist()
        print('\nDiscrete band')
        zi = 0
        for z in reversed(allz):
            if zi >= z:
                lastzi = zi
                zi = compute_detached_tr(ers, z, t0, nr, matrix, maxsteps=maxsteps)
                if np.abs(mp.re(lastzi) - mp.re(zi)) > eps:
                    allzlist.append(mp.re(zi))
                    # to avoid considering same solution twice
                    rholist.append(0.05)
                    print('\rfound isolated eigenvalue zi=%g' % (zi.real))

        return allzlist, rholist
    else:
        return allz, rho


In [ ]:
def run_test(ers,nr,reps,matrix):
    b = len(nr)
    nrns = np.reshape(np.kron(nr, nr), [b, b])

    if matrix is 'adjacency':
        eigs = np.array([scipy.linalg.eigvalsh(hierarchical_random_graph2(ers, nr)) for i in range(0, reps)]).flatten()
    elif matrix is 'laplacian':
        eigs = np.array([scipy.linalg.eigvalsh(GL(hierarchical_random_graph2(ers, nr))) for i in range(0, reps)]).flatten()
    elif matrix is 'norm_laplacian':
        eigs = np.array([scipy.linalg.eigvalsh(NGL(hierarchical_random_graph2(ers, nr))) for i in range(0, reps)]).flatten()

    zmin, zmax = eigs.min(), eigs.max()
    return eigs

In [ ]:
ers = np.array([[50000, 25000], [25000, 5000]]) * 5
nr = [512, 256]

eigs = run_test(ers=ers, nr=nr, reps=20, matrix='adjacency')
plt.hist(eigs[eigs<100], density=1, bins=200)
plt.show()

In [ ]:
#np.savetxt('ers.dat',ers)
#np.savetxt('nrns.dat',nrns)
#np.savetxt('eigs.dat',eigs)
allz, rho = compute_rho(np.linspace(-50,50, 200), ers, nr, 'adjacency', eps=mp.mpf(1E-12), t0=[mp.mpc('0')] * len(nr), include_isolated=False)
# save for further analyses
#np.savetxt('allz.dat',allz)
#np.savetxt('rho.dat',rho)

In [ ]:
plt.hist(eigs[eigs<100], density=1, bins=200)
#plt.vlines(x=zmin, ymin=0, ymax=0.05, color='b')
#plt.vlines(x=zmax, ymin=0, ymax=0.05, color='b')
plt.plot(allz, np.abs(rho))
plt.show()

## Plot them separately, together with their spectra

In [ ]:
plt.figure(figsize=(24,8))
plt.subplot(2,3,1)
plt.imshow(hierarchical_random_graph(M, nr))
plt.grid(False)

plt.subplot(2,3,2)
plt.imshow(M)
plt.grid(False)

plt.subplot(2,3,3)
plt.hist(a,200)

plt.subplot(2,3,4)
plt.hist(l,200)

plt.subplot(2,3,6)
plt.hist(nl,200)
plt.show()

## Define the functions to compute the spectral density

In [ ]:
def compute_tr(z, t0, sigma2, nr, pr_, tol=1E-16, maxsteps=100):
    b = sigma2.shape[0]
    if len(nr) != b:
        raise 'sigma2rs and nr must have same dimensions'
    
    # Define the variables $t_r(z)$ which are the Stieltjes transforms to look for
    tr_ = [sp.symbols('t'+str(i),complex=True) for i in range(0,b)]
    Eq2 = []
    for r in range(0,b):
        c_ = list(pr_[r].free_symbols)[0]
        sum_sigma2rs_ns_ts = np.array([sigma2[r,s]*nr[s]*tr_[s] for s in range(0,b)]).sum()
        #print(sum_sigma2rs_ns_ts)
        fund_eq = tr_[r] -  sp.summation( pr_[r]/(z - c_ - sum_sigma2rs_ns_ts ) , (c_,0,sp.oo))
        # Simplify the fundamental Equation
        fund_eq = sp.simplify(fund_eq)
        # Generate its lambdified expression, for evaluation with mpmath
        F = sp.lambdify(tr_,fund_eq.replace('exp_polar','exp'), modules='mpmath')
        # Append in the list of equations to be solved simulateously
        Eq2.append(F)
    # Now look for the solutions with respect to [t0,t1,...,tb] at this given z
    t = mp.findroot(Eq2, x0=t0, solver='muller', tol=tol, maxsteps=maxsteps)    
    return [t[i] for i in range(0,b)]

def compute_detached_tr(M, z0, t0, nr, pr_, eps=1E-12,maxsteps=100):
    b = M.shape[0]
    if len(nr) != b:
        raise 'Incompatible size of matrix M and total number of blocks'
    IB = sp.eye(b)
    NB = sp.diag(*nr)
    def TzB(z):
        val = sp.diag(*compute_tr(z+eps*1j, t0, M, nr, pr_))
        return val
    
    def eq4(z): # Equation 4 of Peixoto paper
        det = sp.simplify((IB - TzB(z)*M*NB).det())
        return mp.mpc(det)
    
    def trsum(z):
        tr = compute_tr(z+eps*1j, t0, M, nr, [sp.KroneckerDelta(sp.Symbol('c',integer=True),0)] * b)
        return np.sum([t.imag for t in tr])
    z_detached =  mp.findroot(lambda z : eq4(z), x0= mp.mpc(z0), solver='muller', verbose=False, maxsteps=maxsteps)
    return z_detached

def compute_rho(allz, M, nr, matrix, eps=1E-12, maxsteps=150, include_isolated=False, t0=None):
    import time
    from etaprogress.progress import ProgressBar

    bar = ProgressBar(len(allz), max_width=40)
    
    allzlist = allz.tolist()
    rho = np.zeros_like(allz) # initialize rho as zeros
    rholist = rho.tolist()
    b = M.shape[0]
    if t0 is None:
        t0 = [0 + 0.001j] * b # use a default initial value for the stieltjes transform tr
    pr = None
    c = sp.Symbol('c',integer=True) # symbolic variable to sum over in the fundamental equation
    N = np.sum(nr) # total number of nodes
    nrns = np.array([ [(nr[r]*(nr[s]-1)) if r==s else nr[r]*nr[s] for s in range(0,b)] for r in range(0,b)])
    ers = np.multiply(M,nrns) # elementwise multiplication
    dr = np.diagonal(np.diag(nr)*M)
    # Define the pr variable depending on the matrix to investigate
    if matrix is 'laplacian':
        pr = [sp.exp(-dr[r]) * (dr[r]**c) / sp.gamma(c + 1) for r in range(0,b)]
    elif matrix is 'adjacency':
        pr = [sp.KroneckerDelta(c,0) for i in range(0,b)]
    elif matrix is 'norm_laplacian':
        pr = [sp.KroneckerDelta(c,1) for i in range(0,b)]

    print('Continuos band')
    for i,z in enumerate(allz):
        bar.numerator = i+1
        #print('\r',bar,end='')
        t = compute_tr(z + eps*1j, t0, M, nr, pr)
        t0 = t
        rho[i] = -(1.0 / (N*np.pi)) * np.sum([nr[r]*(t[r].imag) for r in range(0,b)])
        print('\r%s Percent done=%.1f %%\tz=%.2f\trho(z)=%g\t%s' % (bar,float(i+1)/len(allz)*100,z,rho[i],t), end='')

    # Then look for the detached eigenvalues by looking for the z such that it solves Eq.4=0
    # where rho is 0 look for separate eigenvalues
    if include_isolated:
        zi,ti = None,None
        print('\nDiscrete band')
        zi = mp.mpc(np.min(allz))
        for z in allz:
            if mp.re(zi) <= z:
                lastzi = zi
                zi = compute_detached_tr(M, z, [1E-5j] * b, nr, pr, maxsteps=maxsteps)
                if np.abs(mp.re(lastzi)-mp.re(zi)) > eps:
                    allzlist.append(mp.re(zi))
                    rholist.append(0.05) # to avoid considering same solution twice
                    print('found isolated eigenvalue zi=%g' % (zi.real))

        return allzlist, rholist
    else:
        return allz, rho

## Compute adjacency spectrum

In [ ]:
allz, rho = compute_rho(np.linspace(-70,70,10), M, nr, 'adjacency', eps=1E-6, maxsteps=300, include_isolated=False)

# Plot adjacency spectrum

In [ ]:
plt.figure(figsize=(12,8))
x = plt.hist(a,bins=500, density=True)
norm_factor = 1
plt.plot(allz/norm_factor,rho*norm_factor)
plt.show()

## Compute Laplacian spectrum

In [ ]:
allz, rho = compute_rho(np.linspace(0,300,50), M, nr, 'laplacian', eps=1E-9, maxsteps=200, include_isolated=False,t0=[(0+1j)*1E-6]*len(nr))

In [ ]:
plt.figure(figsize=(12,8))
#x = plt.hist(l,bins=500, density=True)
norm_factor = 1
plt.plot(allz/norm_factor,rho*norm_factor)
plt.show()

In [ ]:
plt.plot(allz,rho)